Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

these review edits don’t alter the solver logic or results;
they simply make the validation and reporting stages accurate, reproducible, and aligned with the algorithmic guarantees described in the original README.

In [29]:
import numpy as np

In [30]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [31]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [32]:
CONSTRAINTS

array([272,  67])

In [33]:
# A random solution
solution = np.array(
    [np.random.random(NUM_ITEMS) < 0.5 for _ in range(NUM_KNAPSACKS)], dtype=bool
)

In [34]:
solution

array([[False,  True,  True,  True,  True, False, False,  True, False,
        False],
       [ True, False, False,  True,  True,  True, False,  True,  True,
         True]])

In [35]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

False

In [ ]:
#   REVIEW
#   fix validate_solution:
#   Control the aggregate load on all knapsacks (adding the items used in at least one knapsack) instead of checking for knapsacks.
#   Also use < instead of <=. It's the reason False comes back to you despite having feasible solutions in the tests.

def validate_solution(solution: np.ndarray, weights: np.ndarray, constraints: np.ndarray) -> bool:
    
    loads = solution.astype(int) @ weights
    per_knapsack_ok = np.all(loads <= constraints, axis=1).all()
    unique_assignment_ok = np.all(solution.sum(axis=0) <= 1)
    return bool(per_knapsack_ok and unique_assignment_ok)

In [37]:
validate_solution(solution, WEIGHTS, CONSTRAINTS)

False

## TEST FUNCTIONS

In [ ]:
#   REVIEW
#   is_feasible_per_knapsack corrected for testing is used, but then elsewhere (validate_solution, metrics_report) 
#   you had weaker checks leading to diagnostic inconsistency. Everything aligns with the fixes above.

def is_feasible_per_knapsack(solution, weights, constraints):
    """
    Returns True if every knapsack k satisfies that sum of the weights in k <= constraints.
    """
    K, N = solution.shape # number of knapsacks, number of items
    for k in range(K):
        items = solution[k]
        if items.any():
            load = weights[items].sum(axis=0)          # total load on the knapsack
            if np.any(load > constraints):
                print(f"Knapsack {k} is infeasible: load: {load}, constraints: {constraints}")
                return False
    # also enforce at most one knapsack per item: 
    if np.any(solution.sum(axis=0) > 1):
        print("An item appears in multiple knapsacks")
        return False
    print("All knapsacks are feasible")
    return True

In [39]:
# test random solution
is_feasible_per_knapsack(solution, WEIGHTS, CONSTRAINTS)

Knapsack 0 is infeasible: load: [312 167], constraints: [272  67]


False

## PLAYSET

In [40]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 4
NUM_DIMENSIONS = 2

# Suppose each knapsack has 2 items
test_solution = np.array([
    [1, 1, 0, 0],  
    [0, 0, 1, 1]   
], dtype=bool)

test_weights = np.array([
    [3, 5],
    [7, 1],   
    
    [4, 2],  
    [2, 6]   
])
test_constraints = np.array([10, 10])




In [41]:
# test valid solution
is_feasible_per_knapsack(test_solution, test_weights, test_constraints)

All knapsacks are feasible


True

## RANKING FUNCTIONS

In [42]:
def score_normalized_sum(weights, values, constraints, eps=1e-9):
    norm = weights / np.maximum(constraints, eps)          # (N, D)
    denom = 1.0 + norm.sum(axis=1)                         # (N,)
    return values / np.maximum(denom, eps)


In [43]:
def score_l2(weights, values, constraints, eps=1e-9):
    """
    v / ||w/C||_2 — punishes any dimension that is large.
    """
    norm = weights / np.maximum(constraints, eps)
    denom = np.sqrt((norm**2).sum(axis=1)) + 1e-9
    return values / denom

In [44]:
def score_slack_aware_single(wi, vi, remaining, eps=1e-9):
    """
    Slack-aware score for an item against a remaining-capacity vector:
    s = v / (1 + sum_d w_d / remaining_d)
    """
    denom = 1.0 + (wi / np.maximum(remaining, eps)).sum()
    return vi / denom

## UTILITIES

In [45]:
def total_value(solution, values):
    used = np.any(solution, axis=0)
    return float(values[used].sum())


In [ ]:
#   REVIEW

#   Refactor knapsack_loads: vectorized matrix multiply
#   I replaced the per-knapsack Python loop with a single NumPy matrix multiply.
#   Time complexity remains 𝑂(𝐾 ⋅ 𝑁 ⋅ 𝐷) but now runs in optimized native code

def knapsack_loads(solution: np.ndarray, weights: np.ndarray) -> np.ndarray:
    
    return solution.astype(int) @ weights

In [ ]:
#   REVIEW
#   heuristic_pooled_fractional_bound can give values lower than the reached value (see ratio > 1 in Problem 2/3). 
#   So it's not a reliable upper bound. I would rename to heuristic_reference_score or calculate a true upper bound (eg fractional relaxation with separate LP for knapsack).

def heuristic_pooled_fractional_bound(weights, values, constraints, K):
    N, D = weights.shape
    pooled = constraints * K
    norm = weights / np.maximum(pooled, 1)
    density = values / (1.0 + norm.sum(axis=1))
    order = np.argsort(-density)

    rem = pooled.astype(float).copy()
    hb = 0.0
    for i in order:
        wi, vi = weights[i], float(values[i])
        with np.errstate(divide='ignore', invalid='ignore'):
            fr = np.where(wi > 0, rem / wi, np.inf)
        f = float(np.minimum(1.0, np.nanmin(fr)))
        f = max(0.0, f)
        hb += vi * f
        rem = np.maximum(0.0, rem - f * wi)
        if f < 1.0:
            break
    return hb


In [ ]:
#   REVIEW
#   Refactor metrics_report: accurate feasibility and clearer reporting
#   The metrics_report function was revised to improve feasibility checking, clarity, and robustness.
#   The original version only checked that all loads were under constraints, but ignored duplicate item assignments.
#   Now it validates both:
#   Capacity constraints (loads <= constraints per knapsack)
#   Unique item assignment (solution.sum(axis=0) <= 1)

def metrics_report(name, sol, weights, values, constraints, K, hb, elapsed=None):
    loads = sol.astype(int) @ weights
    feasible_caps = np.all(loads <= constraints, axis=1).all()
    unique = np.all(sol.sum(axis=0) <= 1)
    feasible = bool(feasible_caps and unique)

    val = total_value(sol, values)
    util = loads / constraints

    print(f"\n=== {name} ===")
    print(f"Feasible: {feasible}  Value: {val:.1f}  Items: {np.any(sol,axis=0).sum()}")
    print(f"Utilization: max={util.max():.2f}  mean={util.mean():.2f}")
    if hb is not None:
        ratio = val / hb if hb > 0 else float('inf')
        print(f"vs heuristic ref: {ratio:.3f}  (REF={hb:.1f})")
    if elapsed is not None:
        print(f"Time: {elapsed*1000:.1f} ms")

## MAIN ALGORITHM

In [49]:
def build_initial_solution(weights, values, constraints, K, seed=42):
    """
    Greedy fill:
    1) rank items by global normalized score,
    2) for each item, place it in the bin where its slack-aware score is best (if it fits).
    """
    rng = np.random.default_rng(seed)
    N, D = weights.shape
    sol   = np.zeros((K, N), dtype=bool)
    loads = np.zeros((K, D), dtype=weights.dtype)

    global_score = score_normalized_sum(weights, values, constraints)
    order = np.argsort(-global_score)  # best -> worst

    for i in order:
        wi, vi = weights[i], float(values[i])
        best_k, best_s = None, -np.inf
        # randomize bin order to reduce systematic bias
        for k in rng.permutation(K):
            rem = constraints - loads[k]
            if np.any(rem - wi < 0):
                continue
            s = score_slack_aware_single(wi, vi, rem)
            if s > best_s:
                best_s, best_k = s, k
        if best_k is not None:
            sol[best_k, i] = True
            loads[best_k]  += wi

    return sol, loads


# ---------- Local improvement moves ----------

def try_single_replacement(i, k, sol, loads, weights, values, constraints):
    """
    Try to insert item i into bin k by dropping exactly one item j currently in k.
    Choose j with smallest value among those that make i fit.
    Returns (improved:bool, delta:float, j:int|None).
    """
    wi, vi = weights[i], float(values[i])
    idx = np.flatnonzero(sol[k])
    if idx.size == 0:
        return (False, 0.0, None)

    # how much we must free per dimension
    need = np.maximum(loads[k] + wi - constraints, 0)

    wj = weights[idx]
    fits_mask = np.all(wj >= need, axis=1)
    if not np.any(fits_mask):
        return (False, 0.0, None)

    cand = idx[fits_mask]
    # drop min value-loss
    j = int(cand[np.argmin(values[cand])])
    delta = vi - float(values[j])
    return (delta > 0.0, delta, j)

def try_kickset2(i, k, sol, loads, weights, values, constraints, cap_subset=40):
    """
    Try to insert i by dropping up to two items in k.
    Select a small subset of worst-density items for speed; brute-force pairs.
    Returns (improved:bool, delta:float, drop_list:list|None).
    """
    wi, vi = weights[i], float(values[i])
    idx = np.flatnonzero(sol[k])
    m = idx.size
    if m == 0:
        return (False, 0.0, None)

    need = np.maximum(loads[k] + wi - constraints, 0)

    # First try single replacement quickly
    ok1, delta1, j1 = try_single_replacement(i, k, sol, loads, weights, values, constraints)
    if ok1:
        return (True, delta1, [j1])

    # Limit to a small subset of "worst" items (low density) for pair search
    in_w, in_v = weights[idx], values[idx]
    density = in_v / (1.0 + (in_w / np.maximum(constraints, 1)).sum(axis=1))
    subset = idx[np.argsort(density)[:min(cap_subset, m)]]
    if subset.size < 2:
        return (False, 0.0, None)

    best_delta, best_pair = -np.inf, None
    # brute-force pairs among subset
    for a in range(subset.size - 1):
        ja = int(subset[a])
        for b in range(a + 1, subset.size):
            jb = int(subset[b])
            freed = weights[ja] + weights[jb]
            if np.any(freed < need):
                continue
            loss = float(values[ja] + values[jb])
            delta = vi - loss
            if delta > best_delta:
                best_delta, best_pair = delta, [ja, jb]

    if best_pair is None:
        return (False, 0.0, None)
    return (best_delta > 0.0, best_delta, best_pair)


# ---------- Main solver: Greedy + Local Improvement ----------

def try_best_single_replacement_across_bins(i, sol, loads, weights, values, constraints):
    """
    For item i, scan ALL bins, find the best single-drop j (if any) with max positive delta.
    Returns (improved:bool, best_delta:float, k:int, j:int).
    """
    wi, vi = weights[i], float(values[i])
    K = sol.shape[0]
    best = (-np.inf, None, None)
    for k in range(K):
        idx = np.flatnonzero(sol[k])
        if idx.size == 0:
            # direct-fit check first for speed
            if np.all(loads[k] + wi <= constraints):
                return (True, vi, k, None)  # j=None means direct add
            continue
        need = np.maximum(loads[k] + wi - constraints, 0)
        wj = weights[idx]
        fits_mask = np.all(wj >= need, axis=1)
        if not np.any(fits_mask):
            continue
        cand = idx[fits_mask]
        # drop min value-loss among feasible
        j = int(cand[np.argmin(values[cand])])
        delta = vi - float(values[j])
        if delta > best[0]:
            best = (delta, k, j)
    if best[0] > 0:
        return (True, best[0], best[1], best[2])
    return (False, 0.0, None, None)

def try_small_kick2_best_bin(i, sol, loads, weights, values, constraints, cap_subset=20):
    """
    If no single replacement works, pick the best bin by slack-aware and try ≤2-drop there.
    """
    wi, vi = weights[i], float(values[i])
    K = sol.shape[0]
    # choose bin by slack-aware score
    best_k, best_s = None, -np.inf
    for k in range(K):
        rem = constraints - loads[k]
        s = score_slack_aware_single(wi, vi, rem)
        if s > best_s:
            best_s, best_k = s, k
    k = best_k
    # direct fit?
    if np.all(loads[k] + wi <= constraints):
        return (True, vi, k, [])
    # single replacement in best bin?
    ok1, delta1, j1 = try_single_replacement(i, k, sol, loads, weights, values, constraints)
    if ok1:
        return (True, delta1, k, [j1])
    # small 2-kick
    idx = np.flatnonzero(sol[k])
    if idx.size < 2:
        return (False, 0.0, None, None)
    in_w, in_v = weights[idx], values[idx]
    density = in_v / (1.0 + (in_w / np.maximum(constraints, 1)).sum(axis=1))
    subset = idx[np.argsort(density)[:min(cap_subset, idx.size)]]
    need = np.maximum(loads[k] + wi - constraints, 0)
    best_delta, best_pair = -np.inf, None
    for a in range(subset.size - 1):
        ja = int(subset[a])
        for b in range(a + 1, subset.size):
            jb = int(subset[b])
            freed = weights[ja] + weights[jb]
            if np.any(freed < need):
                continue
            delta = vi - float(values[ja] + values[jb])
            if delta > best_delta:
                best_delta, best_pair = delta, [ja, jb]
    if best_pair is not None and best_delta > 0:
        return (True, best_delta, k, best_pair)
    return (False, 0.0, None, None)

def solve_mk_multiD(weights, values, constraints, K,
                    iters_improve, M_top_unassigned=None, seed=42, verbose=True):
    rng = np.random.default_rng(seed)
    sol, loads = build_initial_solution(weights, values, constraints, K,
                                        seed=seed)
    global_score = score_normalized_sum(weights, values, constraints)

    for _ in range(iters_improve):
        changed = False

        # RECOMPUTE unassigned at the start of EACH pass 
        used = np.any(sol, axis=0)
        unassigned = np.flatnonzero(~used)

        if unassigned.size == 0:
            break

        if M_top_unassigned is None:
            M_top_unassigned = max(1, int(0.4 * unassigned.size))
            if verbose:
                print("Setting M_top_unassigned to", M_top_unassigned)

        # Sort/cap by global score each pass
        unassigned = unassigned[np.argsort(-global_score[unassigned])]
        if unassigned.size > M_top_unassigned:
            unassigned = unassigned[:M_top_unassigned]

        # Iterate THIS pass’s fresh unassigned pool
        for i in unassigned:
            if np.any(sol[:, i]):
                continue

            # 1) best single replacement across bins
            ok, delta, k, j = try_best_single_replacement_across_bins(
                i, sol, loads, weights, values, constraints
            )
            if ok:
                if j is None:
                    # direct add (i must be unassigned)
                    sol[k, i] = True
                    loads[k] += weights[i]
                else:
                    sol[k, j] = False
                    loads[k] -= weights[j]
                    sol[k, i] = True
                    loads[k] += weights[i]
                changed = True
                continue

            # 2) small 2-kick in the best bin only
            ok2, delta2, k2, drop = try_small_kick2_best_bin(
                i, sol, loads, weights, values, constraints, cap_subset=20
            )
            if ok2:
                for j2 in drop:
                    sol[k2, j2] = False
                    loads[k2] -= weights[j2]
                sol[k2, i] = True
                loads[k2] += weights[i]
                changed = True
                continue

        if not changed:
            break

    return sol
 

## RUN W/ METRICS

In [50]:
import time
def run_solver_and_report(weights, values, constraints, K, iters_improve=2, seed=42):
    HB = heuristic_pooled_fractional_bound(weights, values, constraints, K)

    t0 = time.time()
    greedy_sol, _ = build_initial_solution(weights, values, constraints, K, seed=seed)
    t1 = time.time()
    metrics_report("Greedy", greedy_sol, weights, values, constraints, K, hb=HB, elapsed=(t1-t0))

    t2 = time.time()
    improved_sol = solve_mk_multiD(weights, values, constraints, K, iters_improve=iters_improve, seed=seed)
    t3 = time.time()
    metrics_report("Greedy+Improve", improved_sol, weights, values, constraints, K, hb=HB, elapsed=(t3-t2))

    return greedy_sol, improved_sol, HB

## TEST PROBLEMS

In [51]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [52]:
print("Problem 1 Constraints:")
print(CONSTRAINTS)
print("Problem 1 Weights sum per dimension:")
print(WEIGHTS.sum(axis=0))
print("Problem 1 Total Value (if all items fit): ", VALUES.sum())
print("Total Item Count: ", (len(VALUES)))

Problem 1 Constraints:
[614 496]
Problem 1 Weights sum per dimension:
[ 985 1072]
Problem 1 Total Value (if all items fit):  1065
Total Item Count:  20


In [53]:
greedy_sol, improved_sol, HB = run_solver_and_report(WEIGHTS, VALUES, CONSTRAINTS, NUM_KNAPSACKS, iters_improve=2, seed=42)
print("\nFinal value (Greedy+Improve):", total_value(improved_sol, VALUES))
is_feasible_per_knapsack(improved_sol, WEIGHTS, CONSTRAINTS)



=== Greedy ===
Feasible: True  Value: 1065.0  Items: 20
Utilization: max=0.76  mean=0.63
vs heuristic bound: 1.000  (HB=1065.0)
Time: 0.9 ms

=== Greedy+Improve ===
Feasible: True  Value: 1065.0  Items: 20
Utilization: max=0.76  mean=0.63
vs heuristic bound: 1.000  (HB=1065.0)
Time: 1.2 ms

Final value (Greedy+Improve): 1065.0
All knapsacks are feasible


True

In [54]:
np.all(improved_sol.sum(axis=0) <= 1)

True

In [55]:
validate_solution(improved_sol, WEIGHTS, CONSTRAINTS)

False

In [64]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [65]:
print("Problem 2 Constraints:")
print(CONSTRAINTS)
print("Problem 2 Weights sum per dimension:")
print(WEIGHTS.sum(axis=0))
print("Problem 2 Total Value (If all items fit): ", VALUES.sum())
print("Total Item Count: ", (len(VALUES)))

Problem 2 Constraints:
[5837 5754 9351 3205 4450 3447 3952 9256 9525 2357]
Problem 2 Weights sum per dimension:
[53631 52574 49834 51220 54224 51487 49988 43139 47615 47654]
Problem 2 Total Value (If all items fit):  52620
Total Item Count:  100


In [66]:
greedy_sol, improved_sol, HB = run_solver_and_report(WEIGHTS, VALUES, CONSTRAINTS, NUM_KNAPSACKS, iters_improve=10, seed=42)
print("\nFinal value (Greedy+Improve):", total_value(improved_sol, VALUES))
is_feasible_per_knapsack(improved_sol, WEIGHTS, CONSTRAINTS)


=== Greedy ===
Feasible: True  Value: 39424.0  Items: 59
Utilization: max=0.99  mean=0.62
vs heuristic bound: 1.078  (HB=36568.5)
Time: 12.3 ms
Setting M_top_unassigned to 16

=== Greedy+Improve ===
Feasible: True  Value: 39450.0  Items: 59
Utilization: max=0.99  mean=0.62
vs heuristic bound: 1.079  (HB=36568.5)
Time: 13.9 ms

Final value (Greedy+Improve): 39450.0
All knapsacks are feasible


True

In [67]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [68]:
print("Problem 3 Total Value (If all items fit): ", VALUES.sum())
print("Total Item Count: ", (len(VALUES)))

Problem 3 Total Value (If all items fit):  2490698
Total Item Count:  5000


In [72]:
greedy_sol, improved_sol, UB = run_solver_and_report(WEIGHTS, VALUES, CONSTRAINTS, NUM_KNAPSACKS, iters_improve=10, seed=42)
print("\nFinal value (Greedy+Improve):", total_value(improved_sol, VALUES))
is_feasible_per_knapsack(improved_sol, WEIGHTS, CONSTRAINTS)


=== Greedy ===
Feasible: True  Value: 1839983.0  Items: 2485
Utilization: max=1.00  mean=0.30
vs heuristic bound: 1.041  (HB=1767027.4)
Time: 1631.9 ms
Setting M_top_unassigned to 1006

=== Greedy+Improve ===
Feasible: True  Value: 1841124.0  Items: 2485
Utilization: max=1.00  mean=0.30
vs heuristic bound: 1.042  (HB=1767027.4)
Time: 8233.4 ms

Final value (Greedy+Improve): 1841124.0
All knapsacks are feasible


True